# 0) Data Loading

In [1]:
import pandas as pd

# Data files path
file1 = "../data/dft-road-casualty-statistics-casualty-2022.csv"
file2 = "../data/dft-road-casualty-statistics-collision-2022.csv"
file3 = "../data/dft-road-casualty-statistics-vehicle-2022.csv"

# Reading CSV files into DataFrames
df_casualty = pd.read_csv(file1)
df_collision = pd.read_csv(file2)
df_vehicle = pd.read_csv(file3)

C:\Users\ATAUMAR\AppData\Local\Temp\ipykernel_9732\1912417982.py:9: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_casualty = pd.read_csv(file1)
C:\Users\ATAUMAR\AppData\Local\Temp\ipykernel_9732\1912417982.py:10: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_collision = pd.read_csv(file2)
C:\Users\ATAUMAR\AppData\Local\Temp\ipykernel_9732\1912417982.py:11: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vehicle = pd.read_csv(file3)


In [2]:
# Printing column names of each DataFrame
print("Column names of 'dft-road-casualty-statistics-casualty-2022.csv':")
print(df_casualty.columns)
print("\nColumn names of 'dft-road-casualty-statistics-collision-2022.csv':")
print(df_collision.columns)
print("\nColumn names of 'dft-road-casualty-statistics-vehicle-2022.csv':")
print(df_vehicle.columns)

Column names of 'dft-road-casualty-statistics-casualty-2022.csv':
Index(['accident_index', 'accident_year', 'accident_reference',
       'vehicle_reference', 'casualty_reference', 'casualty_class',
       'sex_of_casualty', 'age_of_casualty', 'age_band_of_casualty',
       'casualty_severity', 'pedestrian_location', 'pedestrian_movement',
       'car_passenger', 'bus_or_coach_passenger',
       'pedestrian_road_maintenance_worker', 'casualty_type',
       'casualty_home_area_type', 'casualty_imd_decile', 'lsoa_of_casualty'],
      dtype='object')

Column names of 'dft-road-casualty-statistics-collision-2022.csv':
Index(['accident_index', 'accident_year', 'accident_reference',
       'location_easting_osgr', 'location_northing_osgr', 'longitude',
       'latitude', 'police_force', 'accident_severity', 'number_of_vehicles',
       'number_of_casualties', 'date', 'day_of_week', 'time',
       'local_authority_district', 'local_authority_ons_district',
       'local_authority_highway', 'fi

In [3]:
# Merging datasets on 'accident_index'
df_merged = pd.merge(df_casualty, df_collision, on='accident_index')
df_merged = pd.merge(df_merged, df_vehicle, on='accident_index')

print("Merged DataFrame:")
df_merged.head()

Merged DataFrame:


,accident_index,accident_year_x,accident_reference_x,vehicle_reference_x,casualty_reference,casualty_class,sex_of_casualty,age_of_casualty,age_band_of_casualty,casualty_severity,...,sex_of_driver,age_of_driver,age_band_of_driver,engine_capacity_cc,propulsion_code,age_of_vehicle,generic_make_model,driver_imd_decile,driver_home_area_type,lsoa_of_driver
0,2022010352073,2022,10352073,2,1,1,1,17,4,3,...,1,48,8,1461,2,4,RENAULT KANGOO,7,1,E01002687
1,2022010352073,2022,10352073,2,1,1,1,17,4,3,...,1,17,4,125,1,1,HONDA FSH125,1,1,E01001364
2,2022010352573,2022,10352573,1,1,1,2,42,7,3,...,2,42,7,-1,-1,-1,-1,5,1,E01000338
3,2022010352573,2022,10352573,1,1,1,2,42,7,3,...,3,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,2022010352575,2022,10352575,1,1,1,1,20,4,3,...,1,20,4,1896,2,15,VOLKSWAGEN GOLF,8,1,E01000390


In [4]:
df_merged.shape

(168884, 81)

# 1) Basic Data Preparation

In [5]:
# Dropping columns with only one unique value
constant_columns = [col for col in df_merged.columns if df_merged[col].nunique() == 1]
df_merged = df_merged.drop(columns=constant_columns)

In [6]:
# Columns with suffixes '_y'
columns_to_drop = [col for col in df_merged.columns if col.endswith('_y')]

# Dropping all columns with suffixes _y
df_merged.drop(columns=columns_to_drop, inplace=True)

In [7]:
# Checking Nulls
temp = df_merged.isnull().sum()
temp[temp>0]

location_easting_osgr     34
location_northing_osgr    34
longitude                 34
latitude                  34
dtype: int64

In [8]:
# Dropping NAN
df_merged.dropna(inplace=True)

In [ ]:
# Setting date columns format
df_merged['date'] = pd.to_datetime(df_merged['date'], format='mixed')
df_merged['time'] = pd.to_datetime(df_merged['time'], format='%H:%M').dt.time

In [ ]:
# Columns not to encode
special_columns = ["hit_object_off_carriageway", "skidding_and_overturning", "hit_object_in_carriageway", 
 "accident_reference", "special_conditions_at_site", "carriageway_hazards", "speed_limit",
"first_road_number", "accident_index", "casualty_reference"]

# Getting Mapper data 
df_guide = pd.read_excel("../data/dft-road-casualty-statistics-road-safety-open-dataset-data-guide-2023.xlsx", sheet_name="Sheet1")[['field name', 'code/format', 'label']]
temp = df_guide["field name"].value_counts()
temp = temp[temp>2]
temp = temp[temp<100]
temp = temp.drop(special_columns)
temp

In [ ]:
# Encoding Columns values
for column in temp.index:
    if column in df_merged.columns:
        mapper_data = df_guide[df_guide["field name"] == column][['code/format', 'label']].set_index('code/format')
        mapper = mapper_data["label"].to_dict()
        df_merged[column] = df_merged[column].map(mapper)

# 2) Exploratory Data Analysis (EDA)
## 2.1) Basic Exploration

In [ ]:
# Analyze general statistics
print("Statistics of the Numerical Columns:")
display(df_merged.describe())

print("Statistics of the Categorical Columns:")
display(df_merged.describe(include='O'))

### Insights from Statistical Properties

#### Numerical Columns

1. **vehicle_reference_x**:
   - **Mean**: ~1.52
   - **Std. Dev.**: ~0.91
   - Indicates that most accidents involve one vehicle, but there are incidents with multiple vehicles (up to 227 vehicles in one accident).

2. **casualty_reference**:
   - **Mean**: ~1.41
   - **Std. Dev.**: ~2.59
   - Indicates that some accidents involve multiple casualties, with a maximum of 999 casualties in one accident.

3. **age_of_casualty**:
   - **Mean**: ~36.94 years
   - **Std. Dev.**: ~19.20 years
   - Casualties range from very young to very old, with a maximum age of 101.

4. **location_easting_osgr and location_northing_osgr**:
   - **Mean**: ~445930 (easting), ~274609 (northing)
   - **Std. Dev.**: ~91196 (easting), ~122703 (northing)
   - These coordinates indicate the spread of accident locations across the UK.

5. **longitude and latitude**:
   - **Mean**: ~-1.33 (longitude), ~52.36 (latitude)
   - **Std. Dev.**: ~1.33 (longitude), ~1.11 (latitude)
   - These values correspond to locations within the UK.

6. **number_of_vehicles and number_of_casualties**:
   - **Mean**: ~2.18 (vehicles), ~1.77 (casualties)
   - **Std. Dev.**: ~0.91 (vehicles), ~1.33 (casualties)
   - Shows that most accidents involve around 2 vehicles and have around 1-2 casualties.

7. **date**:
   - The mean date is around mid-2022, covering the full year.
   - Accidents are evenly distributed throughout the year.

8. **first_road_number**:
   - **Mean**: ~867
   - **Std. Dev.**: ~1656
   - Road numbers vary widely, indicating accidents occur on many different roads.

9. **speed_limit**:
   - **Mean**: ~36.48 mph
   - **Std. Dev.**: ~14.65 mph
   - Indicates a range of speed limits, with some locations having very high speed limits (up to 70 mph).

10. **age_of_driver**:
    - **Mean**: ~34.77 years
    - **Std. Dev.**: ~21.27 years
    - The average age of drivers involved in accidents, with a wide range from young to elderly drivers.

11. **engine_capacity_cc**:
    - **Mean**: ~1440.78 cc
    - **Std. Dev.**: ~1566.52 cc
    - Indicates a wide range of vehicle engine capacities.

12. **age_of_vehicle**:
    - **Mean**: ~6.67 years
    - **Std. Dev.**: ~6.36 years
    - The age of vehicles involved in accidents varies widely.

#### Categorical Columns

1. **accident_severity**:
   - Most accidents are classified as "Slight", followed by "Serious" and "Fatal".

2. **day_of_week**:
   - Friday is the most common day for accidents, indicating a higher frequency towards the end of the workweek.

3. **time**:
   - The most common time for accidents is around 17:00 (5 PM), suggesting a peak during the evening rush hour.

4. **weather_conditions**:
   - "Fine no high winds" is the most common weather condition, followed by "Raining no high winds", "Snowing no high winds", and "Unknown".

5. **road_surface_conditions**:
   - "Dry" is the most common road surface condition, followed by "Wet/damp".

6. **urban_or_rural_area**:
   - Most accidents occur in urban areas compared to rural areas.

7. **did_police_officer_attend_scene_of_accident**:
   - Police attended the scene in most cases.

8. **vehicle_type**:
   - Cars are thoundation for further analysis, enabling targeted investigation into the factors contributing to road accidents in the UK.ather conditions, and the effectiveness of junction controls on accident rates and severities.